In [4]:
import pandas as pd
import glob
#from afinn import Afinn
from nltk.corpus import stopwords 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
pos_review=(glob.glob("aclImdb\\train\\pos\\*.txt"))

# 긍정,부정 텍스트 읽어오기

lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_pos)

12490

In [6]:
neg_review=(glob.glob("aclImdb\\train\\neg\\*.txt"))

lines_neg=[]
for i in neg_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_neg)

12489

In [14]:
total_text=lines_pos+lines_neg

len(total_text)

24979

In [15]:
x = np.array(["pos", "neg"])
class_Index=np.repeat(x, [len(lines_pos), len(lines_neg)], axis=0)
print(class_Index)

stop_words = stopwords.words('english')
vect = TfidfVectorizer(stop_words=stop_words).fit(total_text) # TFIDF language 모델

['pos' 'pos' 'pos' ... 'neg' 'neg' 'neg']


In [17]:
X_train_vectorized = vect.transform(total_text) # train 데이터안의 단어와 단어의 tfidf값이 들어있음

X_train_vectorized.index = class_Index # X_train_vectorized.index 에 긍정인지 부정인지 넣어주기 위해 (레이블 생성)
print(X_train_vectorized.index)

['pos' 'pos' 'pos' ... 'neg' 'neg' 'neg']


In [9]:
# 로지스틱회귀 모델
from sklearn.linear_model import LogisticRegression,SGDClassifier
model = LogisticRegression() # 괄호 안에 속성을 넣어서 모델을 더 잘 만들 수 있음(과제할 때)
model.fit(X_train_vectorized, class_Index) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
# test가지고 얼마나 잘 예측하는지
pos_review_test=(glob.glob("aclImdb\\test\\pos\\*.txt"))[10]

test=[]
f = open(pos_review_test, 'r')
test.append(f.readlines()[0])
f.close()

In [16]:
predictions = model.predict(vect.transform(test))

predictions[0] # 원래는 array(['pos'], dtype='<U3') 요딴식으로 나오는데 [0]해줘서 pos만 나온듯

'pos'

In [17]:
neg_review_test=(glob.glob("aclImdb\\test\\neg\\*.txt"))[20]

test2=[]
f = open(neg_review_test, 'r')
test2.append(f.readlines()[0])
f.close()

In [18]:
predictions = model.predict(vect.transform(test2))

predictions[0]

'neg'

In [19]:
# 디시젼트리 모델
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier() 
clf.fit(X_train_vectorized, class_Index)

predictions = clf.predict(vect.transform(test))

predictions

array(['pos'], dtype='<U3')

In [20]:
predictions = clf.predict(vect.transform(test2))

predictions

array(['neg'], dtype='<U3')

In [21]:
# SVM 모델
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train_vectorized, class_Index) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
predictions = clf.predict(vect.transform(test))

predictions

array(['pos'], dtype='<U3')

In [23]:
predictions = clf.predict(vect.transform(test2))

predictions

array(['neg'], dtype='<U3')